In [1]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sca_preprocess import trace_categorize
from sca_preprocess import trace_fft, trace_butter_lpf
from sca_preprocess import calc_snr, calc_ttest, calc_sod
from sca_preprocess import select_poi_max_rank, select_poi_threshold
from sca_preprocess import template_build, template_attack, template_attack_report

MCU_freq = 30e6
adc_mul = 4
sample_freq = MCU_freq * adc_mul
cutoff_freq = sample_freq / 8

In [2]:
# sets = [
#     {4, 5},
#     {6, 7},
#     {8, 9, 10, 11},
#     {12, 13, 14, 15},
#     {16, 17, 18, 19, 20, 21, 22},
#     {23, 24, 25, 26, 27, 28, 29, 30, 31}
# ]
# The data in all_traces is arranged in the above order.
mul_all_traces_pre = [np.load(f"./data/fpr_mul_clip_mul_O0_{i}_2000_each.npy") for i in range(5)]

In [3]:
#prepare test dataset:
mul_all_traces = [ele[:,:200,:] for ele in mul_all_traces_pre]
test_traces = [ele[:,-1000:,:] for ele in mul_all_traces_pre]

In [4]:
print(mul_all_traces[0].shape)
print(test_traces[0].shape)

(2, 200, 800)
(2, 1000, 800)


In [5]:
#prepare labels
def construct_mul_label(mul_traces):
    N_class, N_trace, N_sample = mul_traces.shape
    label = []
    tmp = 0
    for i in range(N_class):
        for j in range(N_trace):
            label.append(tmp)
        tmp+=1
    label = np.array(label)
    return label

In [6]:
mul_all_labels=[]
for i in range(5):
    mul_all_labels.append(construct_mul_label(mul_all_traces[i]))
# print(mul_all_labels[0])

In [7]:
test_all_labels=[]
for i in range(5):
    test_all_labels.append(construct_mul_label(test_traces[i]))

In [8]:
#tracewhisperer:
#start cpu cycles idx: 294
#end cpu cycles idx: 477

# start_idx for the collected traces = 280*4
# end_idx for the collected traces = 480*4

mul_all_traces = [ele.reshape(-1,800) for ele in mul_all_traces]
mul_all_traces = [ele[:,14*4:800-3*4] for ele in mul_all_traces]

test_traces = [ele.reshape(-1,800) for ele in test_traces]
test_traces = [ele[:,14*4:800-3*4] for ele in test_traces]
print(mul_all_traces[0].shape)
print(test_traces[0].shape)

(400, 732)
(2000, 732)


In [9]:
mul_all_traces_lpf = [trace_butter_lpf(ele, sample_freq, cutoff_freq) for ele in mul_all_traces]
test_traces_lpf = [trace_butter_lpf(ele, sample_freq, cutoff_freq) for ele in test_traces]

In [10]:
from sklearn.model_selection import train_test_split
train_mul_traces_l=[]
test_mul_traces_l=[]
train_mul_labels_l=[]
test_mul_labels_l=[]
for i in range(len(mul_all_traces_lpf)):
    train_mul_traces, test_mul_traces, train_mul_b, test_mul_b = train_test_split(mul_all_traces_lpf[i], mul_all_labels[i], stratify=mul_all_labels[i], test_size=0.1, shuffle=True, random_state=42)
    train_mul_traces_l.append(train_mul_traces)
    test_mul_traces_l.append(test_mul_traces)
    train_mul_labels_l.append(train_mul_b)
    test_mul_labels_l.append(test_mul_b)

In [11]:
mul_all_traces_dict_list=[]
for i in range(len(train_mul_traces_l)):
    mul_all_traces_dict_list.append(trace_categorize(train_mul_traces_l[i],train_mul_labels_l[i]))

N_poi_mul = 10
poi_space_mul = 40

all_poi_list=[]

for i in range(len(mul_all_traces_dict_list)):
    t_score_mul_i = calc_ttest(mul_all_traces_dict_list[i])
    poi = select_poi_max_rank(t_score_mul_i, N_poi_mul, poi_space_mul)
    all_poi_list.append(poi)

Max t-score: 182.1275
Max t-score: 60.0284
Max t-score: 627.3878
Max t-score: 690.9785
Max t-score: 1669.8443


In [12]:
#validation
#Preliminary verification of template accuracy
template_mul_list=[]
for i in range(len(mul_all_traces_dict_list)):
    template_i = template_build(mul_all_traces_dict_list[i], all_poi_list[i])
    template_mul_list.append(template_i)
    guess_i = template_attack(template_i, test_mul_traces_l[i], test_mul_labels_l[i])
    template_attack_report(guess_i, test_mul_labels_l[i])
print(len(template_mul_list))

#Success: 40
%Success: 100.00%
Error guess: []
Counter: Counter()
#Success: 40
%Success: 100.00%
Error guess: []
Counter: Counter()
#Success: 80
%Success: 100.00%
Error guess: []
Counter: Counter()
#Success: 80
%Success: 100.00%
Error guess: []
Counter: Counter()
#Success: 140
%Success: 100.00%
Error guess: []
Counter: Counter()
5


In [13]:
# test the accuracy of templates
for i in range(len(test_traces)):
    guess_i = template_attack(template_mul_list[i], test_traces_lpf[i], test_all_labels[i])
    template_attack_report(guess_i, test_all_labels[i])

#Success: 2000
%Success: 100.00%
Error guess: []
Counter: Counter()
#Success: 2000
%Success: 100.00%
Error guess: []
Counter: Counter()
#Success: 3999
%Success: 99.98%
Error guess: [2]
Counter: Counter({2: 1})
#Success: 4000
%Success: 100.00%
Error guess: []
Counter: Counter()
#Success: 6982
%Success: 99.74%
Error guess: [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 5, 5, 6, 6, 6]
Counter: Counter({2: 9, 4: 3, 6: 3, 5: 2, 1: 1})
